In [59]:
# 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
import warnings
warnings.filterwarnings('ignore')

# Classifier Libraries
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import collections
from sklearn.metrics import make_scorer, precision_score, recall_score, confusion_matrix, classification_report, matthews_corrcoef, cohen_kappa_score, accuracy_score, average_precision_score, roc_auc_score, precision_recall_fscore_support
from sklearn import metrics
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import  f_classif
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier

# 한글 글씨체 설정
mpl.rc('font', family = 'Malgun Gothic')
# 음수 데이터 표현
mpl.rc('axes', unicode_minus = True)
# 최대로 보일 수 있는 컬럼 설정
pd.set_option('display.max_columns', 118)

In [4]:
# Unique ID -> Actore
df_raw = pd.read_csv("C:/Users/USER/Desktop/pickled/TRAIN/TRAIN.csv", index_col = 'actor_account')
df_raw.drop(columns = ['Unnamed: 0'], inplace = True)
print(df_raw.shape)
df_raw.head()

(5400, 174)


,103,104,110,112,113,114,115,116,118,119,125,126,127,128,129,130,137,138,139,140,141,142,143,145,146,147,148,151,154,155,156,157,158,159,play_time,161,168,169,170,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,198,...,605,606,607,608,609,610,611,612,705,719,720,722,723,log_sum,log_count,log_ip_ratio,per_day,log_ip_perday,quiz_per_playtime,quiz_per_logcount,ply_time_perlog,ply_time_perday,sit_per_time,friends_count,player_kill_count,killed_pc_pertime,killed_npc_pertime,killed_sum,port_pertime,exp_amount,exp_pertime,exp_perlog,151_pertime,151_perlog,get_AP,getap_pertime,getap_perlog,total_AP,totalap_pertime,totalap_perlog,money_get,money_use,money_get_ratio,money_use_ratio,exp_increasingrate,get_item,item_pertime,item_perlog,144,197,max_level,same_ip_rate,friend_action,assist_option,skill_del_or_get,stigmaskill_del_or_get,quest_log_count,about_region,class
actor_account,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000127,10,10,0,0,5,0,0,0,78,78,7,2,0,2,0,0,27,5,32,1,4,70,762,16,0,0,3,0,99,99,191,1,0,1,212,46,19,2,12,0,0,1,1,0,1,0,3,35,207,111,3,3,96,96,1,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,5,224,0,0,0,5215,6.0,6.0,4,10.0,0.0000,0.0000,62.2667,212.0,0.6386,0.0,4.0,1.1032,0.4322,1.5354,1.7399,676474.0,23641.7149,29249.5614,0.0,0.0,173799.0,9483.0794,171018.2000,173799.0,9483.0794,171018.2000,6.1688,2.2269,106.8667,29.5333,-100713.1206,863.0,8.7885,207.6667,0.0,0.0,50.0,1.6667,1,0,73,40,216,0.0,0
1000344,18,17,0,0,1,0,0,0,43,42,6,14,2,12,0,0,24,19,49,0,0,111,1420,36,2,0,18,0,139,139,533,0,0,0,313,70,43,0,18,0,0,53,53,0,9,0,1,138,935,126,13,12,125,124,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,2,585,0,0,0,8144,11.0,6.0,4,18.0,0.0000,0.0000,81.8167,313.0,0.4874,0.0,2.0,0.2621,0.2327,0.4948,1.3453,41062625.0,23004.0585,547273.9751,0.0,0.0,50801.0,1061.6971,25891.7333,50801.0,1061.6971,25891.7333,13.8850,1.4405,308.5667,28.1333,687725.4463,1042.0,15.1698,298.9500,0.0,0.0,50.0,1.6364,3,0,26,102,396,0.0,0
1000779,11,12,0,0,1,0,0,0,84,83,0,0,0,0,0,0,21,18,42,0,0,61,5585,37,0,0,0,0,0,0,0,0,0,0,964,153,0,0,17,0,0,0,0,0,0,0,0,0,9,54,0,0,1107,1107,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,43,0,0,0,66342,4.0,11.0,4,11.0,0.0000,0.0000,489.5000,964.0,0.3494,0.0,0.0,0.0974,0.0689,0.1663,0.2587,17640790.0,300.1102,17773.8447,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,0.0367,0.2340,4.9166,26.3333,69767.0012,22531.0,91.1750,13480.5000,0.0,0.0,35.0,2.7500,0,0,0,0,0,0.0,1
1000949,8,9,0,0,0,0,0,1267,15,12,0,0,0,0,0,0,6,0,6,0,0,20,1,4,0,0,0,0,1640,1639,1,0,0,0,982,153,0,0,4,0,0,0,0,0,0,0,0,0,0,1,0,0,10,10,0,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,25,0,0,0,44151,4.0,8.0,4,8.0,5.0741,0.1138,746.0000,982.0,0.0571,0.0,0.0,0.0256,0.0000,0.0256,0.0928,468.0,0.0057,1.6364,0.0,0.0,13.0,0.0455,13.0000,13.0,0.0455,13.0000,0.0000,0.0035,0.0000,0.5000,1.6364,12201.0,49.1657,9286.5000,0.0,0.0,30.0,2.0000,0,0,0,0,0,0.0,1
1001067,22,20,0,1,9,0,0,0,4657,4643,0,0,0,0,0,0,141,51,193,0,0,186,12101,154,0,0,4,0,0,0,0,0,0,0,1861,296,2,0,116,0,0,0,0,0,0,0,0,0,1591,93,0,0,631,631,1,0,0,0.0,0,...,0,0,0.0,0,0,0.0,0,0.0,0,162,0,0,0,49255,15.0,13.5,10,22.0,0.0000,0.0000,1120.9000,1861.0,20.2832,0.0,0.0,0.5645,0.2066,0.7711,0.7527,54857978.0,2276.6131,94174.2098,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0000,0.0000,7.4506,0.3805,971.7667,62.9667,280159.7287,13408.0,56.9362,8379.2667,0.0,0.0,43.0,1.4667,0,0,142,60,940,0.0,0


In [6]:
lgbc=LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

embeded_lgb_selector = SelectFromModel(lgbc, max_features=90)
embeded_lgb_selector.fit(df_raw[df_raw.columns.difference(['class'])], df_raw['class'])

embeded_lgb_support = embeded_lgb_selector.get_support()
embeded_lgb_feature = df_raw[df_raw.columns.difference(['class'])].loc[:,embeded_lgb_support].columns.tolist()
print(str(len(embeded_lgb_feature)), 'selected features')

35 selected features


['118','119','137','142','143','145','161','168', '169','170','186', '188', '192', '202', '206', '210','268 'exp_pertime', 'item_pertime', 'log_count', 'log_ip_ratio','log_sum','max_level','money_get','money_get_ratio', 'money_use', 'money_use_ratio', 'per_day', 'play_time', 'ply_time_perday', 'ply_time_perlog', 'port_pertime' 'quest_log_count', 'same_ip_rate', 'stigmaskill_del_or_get']

In [30]:
df01 = df_raw[embeded_lgb_feature]

# 중복 데이터 제거
df01.drop(columns = ['money_get','money_use','per_day','142'], axis = 1, inplace = True)
label = pd.read_csv('./labeled_account.csv')
label = label.rename({'account':'actor_account'}, axis = 1)
df02 = pd.merge(df01, label, on = 'actor_account')
df02.set_index('actor_account', inplace = True)
df02.head()

,118,119,137,143,145,161,168,169,170,186,188,192,202,206,210,268,exp_pertime,item_pertime,log_count,log_ip_ratio,log_sum,max_level,money_get_ratio,money_use_ratio,play_time,ply_time_perday,ply_time_perlog,port_pertime,quest_log_count,same_ip_rate,stigmaskill_del_or_get,class
actor_account,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000127,78,78,27,762,16,46,19,2,12,35,111,96,827,15,1,20,23641.7149,8.7885,6.0,6.0,5215,50.0,106.8667,29.5333,212,212.0,62.2667,1.7399,216,1.6667,40,0
1000344,43,42,24,1420,36,70,43,0,18,138,126,124,736,23,3,17,23004.0585,15.1698,11.0,6.0,8144,50.0,308.5667,28.1333,313,313.0,81.8167,1.3453,396,1.6364,102,0
1000779,84,83,21,5585,37,153,0,0,17,0,54,1107,18861,0,75,12,300.1102,91.1750,4.0,11.0,66342,35.0,4.9166,26.3333,964,964.0,489.5000,0.2587,0,2.7500,0,1
1000949,15,12,6,1,4,153,0,0,4,0,1,10,12199,16,0,9,0.0057,49.1657,4.0,8.0,44151,30.0,0.0000,0.5000,982,982.0,746.0000,0.0928,0,2.0000,0,1
1001067,4657,4643,141,12101,154,296,2,0,116,0,93,631,11949,13,0,20,2276.6131,56.9362,15.0,13.5,49255,43.0,971.7667,62.9667,1861,1861.0,1120.9000,0.7527,940,1.4667,60,0


# 모델 생성

### 이상치 데이터 제거(휴먼 기준)

In [43]:
# human의 이상치 찾기
def find_max_outlier_human(d_cp, column):
    # class 0 만
    fraud_column_data = d_cp[d_cp['class']==0][column]
    
    # Q1, Q3
    q_25 = np.percentile(fraud_column_data.values, 25)
    q_75 = np.percentile(fraud_column_data.values, 75)
    
    # 이상치 최대값 계산
    iqr = q_75 - q_25
    iqr = iqr * 1.5
    highest = q_75 + iqr
    outlier_index = fraud_column_data[(fraud_column_data > highest)].index
    
    # 이상치 최대값 반환
    return highest

In [44]:
# 데이터프레임 하나를 가지고 모든 컬럼을 돌면서 컬럼 내 이상치 최대값 찾기
def replace_max_outlier_human(dataframe):
    for col in dataframe.columns:
        # 한 컬럼마다 최대 이상치 값 찾기 위한 함수 호출
        highest = int(find_max_outlier_human(dataframe, str(col)))
        
        # 최대 이상치가 0이면 바꾸지 않는다
        if highest > 0:
            dataframe[str(col)][dataframe[str(col)] > highest] = highest
            
    return dataframe

In [47]:
df03 = df02.copy()
df03 = replace_max_outlier_human(df03)
df03.describe()

,118,119,137,143,145,161,168,169,170,186,188,192,202,206,210,268,exp_pertime,item_pertime,log_count,log_ip_ratio,log_sum,max_level,money_get_ratio,money_use_ratio,play_time,ply_time_perday,ply_time_perlog,port_pertime,quest_log_count,same_ip_rate,stigmaskill_del_or_get,class
count,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5.400000e+03,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000,5400.000000
mean,123.515370,121.644630,18.686111,2570.642778,38.450000,137.651481,20.754259,1.243889,13.264815,133.987778,128.706852,161.699444,1867.010370,32.596852,5.654444,50.771111,-3.177328e+02,239.208870,11.391481,15.115130,18651.784259,39.909444,259.491447,37.560660,554.014074,554.014074,150.528792,1.861448,463.338148,2.206593,124.050000,0.074074
std,135.964257,134.420118,25.935667,2731.102148,43.474868,131.790045,22.841250,1.701344,17.944788,181.930967,126.988163,190.168466,1973.655385,38.442678,7.345156,58.384260,2.086696e+05,304.013053,8.013145,12.771756,19533.409438,12.949206,271.244564,31.578701,524.581685,524.581685,125.605019,1.521419,539.556783,1.228640,156.701288,0.261916
min,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.191209e+07,0.020400,1.000000,1.000000,69.000000,0.000000,0.000000,0.000000,36.000000,36.000000,3.588200,0.000000,0.000000,1.000000,0.000000,0.000000
25%,18.000000,17.000000,0.000000,411.000000,5.000000,33.000000,3.000000,0.000000,0.000000,0.000000,30.000000,18.000000,293.000000,3.000000,0.000000,9.000000,9.472433e+01,20.132925,5.000000,5.500000,3017.750000,32.000000,41.377175,13.373150,115.000000,115.000000,51.979100,0.563375,37.000000,1.250000,11.000000,0.000000
50%,63.000000,62.000000,4.000000,1353.000000,19.000000,92.000000,12.000000,0.000000,4.000000,42.000000,81.000000,77.000000,946.000000,16.000000,2.000000,22.000000,2.794227e+03,87.922400,10.000000,11.000000,9816.000000,47.000000,161.650000,27.750000,361.000000,361.000000,101.866250,1.523800,236.000000,1.777800,50.000000,0.000000
75%,188.250000,185.000000,28.000000,4016.250000,60.000000,192.000000,32.000000,2.000000,20.000000,206.000000,187.000000,241.250000,3128.500000,48.000000,9.250000,72.000000,1.207590e+04,351.034550,16.000000,20.500000,29860.250000,50.000000,379.090475,54.000000,899.250000,899.250000,219.998150,2.863550,716.250000,2.800000,179.250000,0.000000
max,403.000000,399.000000,72.000000,8243.000000,132.000000,472.000000,76.000000,5.000000,50.000000,567.000000,445.000000,618.000000,5385.000000,114.000000,20.000000,185.000000,3.311600e+04,920.000000,35.000000,45.000000,59318.000000,50.000000,858.000000,113.000000,1793.000000,1793.000000,390.000000,6.000000,1797.000000,5.000000,489.000000,1.000000


In [48]:
x_train = df03[df03.columns.difference(['class'])]
y_train = df03['class']

In [49]:
# 스케일 조정
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)


In [64]:
lgbm1 = LGBMClassifier()

In [68]:
n_splits = 20

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm1, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9177183767652318 표준편차 : 0.030151555756534713


In [69]:
n_splits = 10

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm1, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9194117652758559 표준편차 : 0.02281930126918638


In [71]:
lgbm2 = LGBMClassifier(learning_rate = 0.05, num_iterations = 1000, early_stopping_round = 5, max_depth  =10,
                      num_leaves = 10, boosting_type = 'dart', objective = 'binary', metric = 'binary_logloss')

In [72]:
n_splits = 20

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm2, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9213633236377328 표준편차 : 0.05706472549590807


In [73]:
n_splits = 10

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm2, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9287959515216138 표준편차 : 0.03023474781577233


### 이상치 데이터 미제거

In [74]:
df03 = df02.copy()
x_train = df03[df03.columns.difference(['class'])]
y_train = df03['class']

# 스케일 조정
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)

In [75]:
lgbm1 = LGBMClassifier()

In [76]:
n_splits = 20

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm1, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9216929885350937 표준편차 : 0.0383385310940429


In [77]:
n_splits = 10

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm1, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9246062229634642 표준편차 : 0.022096211366239196


In [78]:
lgbm2 = LGBMClassifier(learning_rate = 0.05, num_iterations = 1000, early_stopping_round = 5, max_depth  =10,
                      num_leaves = 10, boosting_type = 'dart', objective = 'binary', metric = 'binary_logloss')

In [79]:
n_splits = 20

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm2, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9255803272908535 표준편차 : 0.04226622949195052


In [80]:
n_splits = 10

kfold = StratifiedKFold(n_splits=n_splits, shuffle=True)
result = cross_val_score(lgbm2, x_train_scaled, y_train, scoring='f1', cv=kfold)

result_mean = result.mean()
result_std = result.std()

print('평균 f1 score : {} 표준편차 : {}'.format(result_mean, result_std))

평균 f1 score : 0.9206896726883402 표준편차 : 0.024849651379174827
